In [2]:
import requests
from env import config

MINDSDB_QUERY_ENDPOINT = "http://127.0.0.1:47334/api/sql/query"




def mindsdb_query(MINDSDB_QUERY_ENDPOINT, sql_query):
    headers = {"Content-Type": "application/json"}
    try:
        response = requests.post(MINDSDB_QUERY_ENDPOINT, json={
                                 "query": sql_query}, headers=headers)
        response.raise_for_status()
        return response
    except requests.exceptions.RequestException as e:
        print(f"An error occurred during the HTTP request: {str(e)}")
        return None

In [2]:
def create_skill():
    sql_query = f""""
CREATE SKILL loan_agent.loan_skill
    USING
    type = 'text_to_sql',
    database = 'test', 
    tables = ['loan_data'];
"""

    res = mindsdb_query(MINDSDB_QUERY_ENDPOINT, sql_query)
    if res is not None:
        if res.status_code == 200:
            print("Skill created successfully")
        else:
            print(
                f"Failed to create skill. HTTP status code: {res.status_code}")
    else:
        print("Failed to create skill. No response received.")

In [6]:
API_KEY = config("API_KEY")

def create_model():
    sql_query = f"""
CREATE MODEL loan_agent.my_model
PREDICT answer
USING
    engine = 'openai',
    input_column = 'question',
    api_key = '{API_KEY}', 
    model_name='gpt-4', 
    mode = 'conversational',
    user_column = 'question' ,
    assistant_column = 'answer',
    max_tokens=100,
    temperature=0,
    verbose=True,
    prompt_template='Answer the user input in a helpful way';


"""
    res = mindsdb_query(MINDSDB_QUERY_ENDPOINT, sql_query)
    if res is not None:
        if res.status_code == 200:
            print("Model created successfully")
        else:
            print(
                f"Failed to create model. HTTP status code: {res.status_code}")
    else:
        print("Failed to create model. No response received.")
    

In [7]:
create_model()

Model created successfully


In [6]:
def create_agent():
    sql_query = f""""
CREATE AGENT loan_agent.my_loan_agent
USING
   model = 'loan_agent.loan_approval_predictor',
   skills = ['loan_agent.loan_skill']; 

"""
    res = mindsdb_query(MINDSDB_QUERY_ENDPOINT, sql_query)
    if res is not None:
        if res.status_code == 200:
            print("Agent created successfully")
        else:
            print(
                f"Failed to create agent. HTTP status code: {res.status_code}")
    else:
        print("Failed to create agent. No response received.")

In [6]:
def create_chatbot():
    sql_query = f"""
CREATE CHATBOT lendswift_chatbot
USING
    database = 'mindsdb_slack', 
    agent = 'my_loan_agent', 
    enable_dms = true;

"""
    res = mindsdb_query(MINDSDB_QUERY_ENDPOINT, sql_query)
    if res is not None:
        if res.status_code == 200:
            print("Chatbot created successfully")
        else:
            print(f"Failed to create chatbot. HTTP status code: {res.status_code}")
        
    else:
        print("Failed to create chatbot. No response received.")



In [7]:
create_chatbot()

Chatbot created successfully
